In [36]:
# All the imports
#import dash, plotly, and dash for jupyter notebook 
import dash
import plotly.express as px
import plotly.graph_objs as go
from jupyter_dash import JupyterDash #for jupyter notebook
import dash_table

#import front end libraries 
import dash_core_components as dcc 
import dash_html_components as html 
import dash_bootstrap_components as dbc 
from dash.dependencies import Input, Output

#other packages 
import pandas as pd
import numpy as np

<ipython-input-36-ff298fea8d4a>:7: UserWarning:


The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`



In [2]:
# Read the data
# Data from https://cran.r-project.org/web/packages/dragracer/index.html
rpdr_c = pd.read_csv('data/rpdr_contestants.csv')
rpdr_cp = pd.read_csv('data/rpdr_contestant_performance.csv')
rpdr_e = pd.read_csv('data/rpdr_episodes.csv')
df = pd.read_csv('data/drag.csv')

In [48]:
# Data Cleaning and Pre-processing before visualizing
# Remove episodes where no ranked competition occurred (i.e. the reunion)
to_remove = rpdr_cp[(rpdr_cp['episode'] == 13) & (rpdr_cp['season'].isin(['S11', 'S12']))]
rpdr_cp = rpdr_cp.drop(to_remove.index)

# Regroup episode rankings
# Making it to the finale is considered a win, OUT = not in episode/or eliminated both included
outcome_map = {'OUT': 'OUT', 'LOST1ST ROUND': 'OUT', 'LOST2ND ROUND': 'OUT', 'LOST3RD ROUND': 'OUT', 'MISSCON': 'OUT',
               'DISQ': 'OUT', 'RTRN': 'OUT', 'SAFE+DEPT': 'OUT', 'BTM': 'BOTTOM', 'TOP2': 'WIN', 'WIN+RTRN': 'WIN'}
rpdr_cp['outcome2'] = rpdr_cp['outcome'].replace(outcome_map)

#filter to winners only 
winners = rpdr_cp[rpdr_cp['rank'] == 1]
#remove index 1706, 676 for seasons where winner did not compete in first episode
winners.drop([676, 1706], axis = 0, inplace = True)

#scale outcomes 
winners['outcome_count'] = winners.groupby('contestant')['participant'].transform('count')
winners['outcome_count_scaled'] = winners['outcome_count']/winners.groupby('contestant')['outcome_count'].transform('sum')


/Users/DhruviNishar/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-48-f3d06a32ce28>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-48-f3d06a32ce28>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
winners

,Unnamed: 0,season,rank,missc,contestant,episode,outcome,eliminated,participant,minichalw,finale,penultimate,outcome2,outcome_count,outcome_count_scaled
0,1,S01,1.0,0.0,BeBe Zahara Benet,1,SAFE,0.0,1,0.0,0,0,SAFE,7,0.142857
9,10,S01,1.0,0.0,BeBe Zahara Benet,2,SAFE,0.0,1,0.0,0,0,SAFE,7,0.142857
18,19,S01,1.0,0.0,BeBe Zahara Benet,3,WIN,0.0,1,0.0,0,0,WIN,7,0.142857
27,28,S01,1.0,0.0,BeBe Zahara Benet,4,HIGH,0.0,1,0.0,0,0,HIGH,7,0.142857
36,37,S01,1.0,0.0,BeBe Zahara Benet,5,BTM,0.0,1,0.0,0,0,BOTTOM,7,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,1811,S12,1.0,0.0,Jaida Essence Hall,9,WIN,0.0,1,0.0,0,0,WIN,12,0.083333
1823,1824,S12,1.0,0.0,Jaida Essence Hall,10,WIN,0.0,1,0.0,0,0,WIN,12,0.083333
1836,1837,S12,1.0,0.0,Jaida Essence Hall,11,BTM,0.0,1,0.0,0,0,BOTTOM,12,0.083333
1849,1850,S12,1.0,0.0,Jaida Essence Hall,12,SAFE,0.0,1,0.0,0,0,SAFE,12,0.083333


In [57]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Img(src='assets/logo.png', height='80px', style={'float': 'left'}),
    html.H6(
        children = 'Queen Performance by Season',
        style = {'textAlign': 'left-center', 
                 'color': '#FF1D8E',
                 'font-family': 'Brush Script MT',
                 'font-size': '55px'}),
    dcc.Dropdown(
        id = 'season_dropdown', clearable = False,
        value = 'S07',
        options = [{'label':i, 'value': i} for i in rpdr_cp['season'].unique()],
        style = {'width': '75%', 
                 'margin': 'auto'}
    ),
    dcc.Graph(
        id='rpdr_graph',
        figure= {}
    ),
    html.H6(
        children = 'Winner Performance in Retrospect',
        style = {'textAlign': 'center', 
                 'color': '#FF1D8E',
                 'font-family': 'Brush Script MT',
                 'font-size': '55px'}),
    dcc.Graph(
        id='winners_graph',
        figure= {}
    )
]
)

@app.callback( 
    [Output('rpdr_graph', 'figure'), Output('winners_graph', 'figure')], 
    [Input('season_dropdown', 'value')]) 
def update_graph(season):
    s_x = rpdr_cp[rpdr_cp['season'] == str(season)]
    fig1 = px.bar(s_x, x='contestant', color = 'outcome2', 
                 labels={'count':'Total performance', 
                         'contestant':'Queens (by Rank)',
                         'outcome2':'Outcome'},
                 color_discrete_sequence=['#636EFA', '#FF6692', '#00CC96', '#FFA15A', '#FFFF7C','#999999'],
                 category_orders={"outcome2": ["WIN", "HIGH", "SAFE", "LOW", "BOTTOM","OUT"]})

    fig1.update_layout(legend_traceorder='normal',
                       xaxis={'categoryorder':'array', 
                            'categoryarray':s_x['contestant'].unique()},
                 plot_bgcolor='#F7DAE4')
    
    fig1.update_traces(marker=dict(line=dict(width=0.7, color='black')))

    fig2 = px.bar(winners, x = 'contestant', y = 'outcome_count_scaled', color = 'outcome2',
            labels={'outcome_count_scaled':'Relative Performance', 
                    'contestant':'Winner (in order of season)',
                    'outcome2':'Outcome'},
                 color_discrete_sequence=['#636EFA', '#FF6692', '#00CC96', '#FFA15A', '#FFFF7C'],
                 category_orders={"outcome2": ["WIN", "HIGH", "SAFE", "LOW", "BOTTOM"]})

    fig2.update_layout(legend_traceorder='normal',
                       plot_bgcolor='#F7DAE4')
    
    fig2.update_traces(marker=dict(line=dict(width=0.7, color='black')))

    return fig1, fig2

if __name__ == '__main__':
    app.run_server(mode = 'inline')

Dash is running on http://127.0.0.1:8050/

